In [17]:
import pandas as pd
import numpy as np
import pyBADA.atmosphere as atm

# 自定义转换函数
class conv:
    @staticmethod
    def ft2m(feet):
        return feet * 0.3048
    
    @staticmethod
    def m2ft(meters):
        return meters / 0.3048
    
    @staticmethod
    def kt2ms(knots):
        return knots * 0.51444
    
    @staticmethod
    def ms2kt(ms):
        return ms / 0.51444

# 创建马赫数和CAS范围
mach_range = np.arange(0.70, 0.86, 0.01)
cas_range = np.arange(250, 351, 10)

# 创建结果矩阵
results = []

# 计算所有组合的过渡高度
for mach in mach_range:
    row = [mach]
    for cas in cas_range:
        cas_ms = conv.kt2ms(cas)
        try:
            crossover_m = atm.crossOver(cas=cas_ms, Mach=mach)
            crossover_ft = conv.m2ft(crossover_m)
            crossover_fl = int(crossover_ft / 100)
            row.append(crossover_fl)
        except:import pandas as pd
import numpy as np
import pyBADA.atmosphere as atm

# 自定义转换函数
class conv:
    @staticmethod
    def ft2m(feet):
        return feet * 0.3048
    
    @staticmethod
    def m2ft(meters):
        return meters / 0.3048
    
    @staticmethod
    def kt2ms(knots):
        return knots * 0.51444
    
    @staticmethod
    def ms2kt(ms):
        return ms / 0.51444

# 创建马赫数和CAS范围
mach_range = np.arange(0.70, 0.86, 0.01)
cas_range = np.arange(250, 351, 10)

# 创建结果矩阵
results = []

# 计算所有组合的过渡高度
for mach in mach_range:
    row = [mach]
    for cas in cas_range:
        cas_ms = conv.kt2ms(cas)
        try:
            crossover_m = atm.crossOver(cas=cas_ms, Mach=mach)
            crossover_ft = conv.m2ft(crossover_m)
            crossover_fl = int(crossover_ft / 100)
            row.append(crossover_fl)
        except:
            row.append(None)  # 处理可能的错误情况
    results.append(row)

# 创建列名
columns = ['Mach'] + [f'{cas}kt' for cas in cas_range]

# 创建DataFrame并显示结果
df = pd.DataFrame(results, columns=columns)
print(df.to_string(index=False))



 Mach  250kt  260kt  270kt  280kt  290kt  300kt  310kt  320kt  330kt  340kt  350kt
  0.7    322    304    287    270    254    237    221    205    190    174    159
  0.7    329    311    294    277    261    245    229    213    197    182    167
  0.7    336    318    301    284    268    252    236    220    205    190    175
  0.7    342    325    308    291    275    259    243    227    212    197    182
  0.7    349    331    315    298    282    266    250    235    219    204    190
  0.8    355    338    321    305    289    273    257    242    227    212    197
  0.8    361    344    328    311    295    279    264    249    234    219    204
  0.8    368    351    334    318    302    286    271    255    241    226    211
  0.8    374    357    340    324    308    293    277    262    247    233    218
  0.8    380    363    347    330    315    299    284    269    254    239    225
  0.8    386    369    353    337    321    305    290    275    261    246    232
  0.

In [10]:
from pyBADA.bada4 import Bada4Aircraft, BADA4
from pyBADA.aircraft import Airplane
import pyBADA.atmosphere as atm
import math
import pandas as pd
import numpy as np

# 自定义转换函数
class conv:
    @staticmethod
    def ft2m(feet):
        return feet * 0.3048
    
    @staticmethod
    def m2ft(meters):
        return meters / 0.3048
    
    @staticmethod
    def kt2ms(knots):
        return knots * 0.51444
    
    @staticmethod
    def ms2kt(ms):
        return ms / 0.51444

def calculate_descent_profile(cruise_fl, target_fl, aircraft_mass, 
                              cruise_mach, descent_mach, high_cas, low_cas=None, 
                              low_cas_fl=100, ac_model="A320-232", bada_version="4.2",
                              filepath="/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models",
                              constant_cas=None):
    """
    计算下降剖面
    
    参数:
    cruise_fl: 巡航高度层 (FL)
    target_fl: 目标高度层 (FL)
    aircraft_mass: 飞机重量 (kg)
    cruise_mach: 巡航马赫数
    descent_mach: 下降马赫数，如果constant_cas不为None则忽略
    high_cas: 过渡高度后的CAS (kt)，如果constant_cas不为None则忽略
    low_cas: 低高度CAS (kt)，如果为None则与high_cas相同
    low_cas_fl: 低高度CAS开始的高度层 (FL)
    ac_model: 飞机型号
    bada_version: BADA版本
    filepath: BADA数据文件路径
    constant_cas: 如果不为None，则使用恒定CAS下降
    
    返回:
    包含下降剖面总结信息的字典
    """
    # 初始化飞机模型
    AC = Bada4Aircraft(
        badaVersion=bada_version,
        acName=ac_model,
        filePath=filepath,
    )
    
    # 创建BADA4性能计算对象
    bada4 = BADA4(AC)
    
    # 设置低高度CAS
    if low_cas is None:
        low_cas = high_cas
    
    # 为恒定CAS下降设置参数
    if constant_cas is not None:
        descent_mach = 0.9  # 设置一个较高的马赫数，确保过渡高度远高于巡航高度
        high_cas = constant_cas
        low_cas = constant_cas if low_cas is None else low_cas
    
    # 计算过渡高度
    high_cas_ms = conv.kt2ms(high_cas)
    crossover_m = atm.crossOver(cas=high_cas_ms, Mach=descent_mach)
    crossover_ft = conv.m2ft(crossover_m)
    crossover_fl = int(crossover_ft / 100)
    
    # 创建速度剖面名称
    if constant_cas is not None:
        if high_cas == low_cas:
            profile_name = f"{high_cas}kt constant"
        else:
            profile_name = f"{high_cas}kt→{low_cas}kt@FL{low_cas_fl}"
    else:
        if high_cas == low_cas:
            profile_name = f"{descent_mach}M/{high_cas}kt"
        else:
            profile_name = f"{descent_mach}M/{high_cas}kt/{low_cas}kt@FL{low_cas_fl}"
    
    def calculate_performance(flight_level, aircraft_mass, DeltaTemp=0):
        """计算指定高度层的下降性能"""
        altitude_ft = flight_level * 100
        altitude_m = conv.ft2m(altitude_ft)
        
        # 计算大气参数
        theta_val, delta_val, sigma_val = atm.atmosphereProperties(altitude_m, DeltaTemp)
        
        # 根据下降速度规则确定速度
        if constant_cas is None and flight_level >= crossover_fl:
            # 使用指定马赫数
            M = descent_mach
            tas = atm.mach2Tas(M, theta_val)
            cas = atm.mach2Cas(M, theta_val, delta_val, sigma_val)
            speed_mode = f"Mach {descent_mach}"
            flight_evolution = "constM"
        elif flight_level >= low_cas_fl:
            # 使用高高度CAS
            cas = conv.kt2ms(high_cas)
            tas = atm.cas2Tas(cas, delta_val, sigma_val)
            M = atm.tas2Mach(tas, theta_val)
            speed_mode = f"CAS {high_cas}kt"
            flight_evolution = "constCAS"
        else:
            # 使用低高度CAS
            cas = conv.kt2ms(low_cas)
            tas = atm.cas2Tas(cas, delta_val, sigma_val)
            M = atm.tas2Mach(tas, theta_val)
            speed_mode = f"CAS {low_cas}kt"
            flight_evolution = "constCAS"
        
        # 动态计算能量分配因子
        esf_des = Airplane.esf(
            h=altitude_m,
            DeltaTemp=DeltaTemp,
            flightEvolution=flight_evolution,
            phase="des",
            M=M
        )
        
        # 计算升力系数
        cl = bada4.CL(delta=delta_val, mass=aircraft_mass, M=M)
        
        # 计算阻力系数(干净构型)
        HLid = 0.0  # 无襟翼
        LG = "LGUP"  # 起落架收起
        cd = bada4.CD(HLid=HLid, LG=LG, CL=cl, M=M)
        
        # 计算阻力
        drag = bada4.D(delta=delta_val, M=M, CD=cd)
        
        # 计算怠速推力
        idle_thrust = bada4.Thrust(
            delta=delta_val,
            theta=theta_val,
            M=M,
            rating="LIDL",  # 怠速设置
            DeltaTemp=DeltaTemp
        )
        
        # 计算燃油流量 (kg/s)
        fuel_flow = bada4.ff(
            delta=delta_val,
            theta=theta_val,
            M=M,
            rating="LIDL",  # 怠速设置
            DeltaTemp=DeltaTemp
        )
        
        # 转换为kg/h
        fuel_flow_kgh = fuel_flow * 3600
        
        # 计算怠速下降率
        idle_descent_rate = bada4.ROCD(
            T=idle_thrust,
            D=drag,
            v=tas,
            mass=aircraft_mass,
            ESF=esf_des,
            h=altitude_m,
            DeltaTemp=DeltaTemp
        )
        
        # 计算下降角(度)
        if tas > 0:
            descent_angle = math.degrees(math.asin(idle_descent_rate / tas))
        else:
            descent_angle = 0
        
        # 计算下降梯度(百分比)
        descent_gradient = 100 * abs(idle_descent_rate / tas) if tas > 0 else 0
        
        # 计算高距离
        ft_per_nm = abs(idle_descent_rate * 196.85 / conv.ms2kt(tas) * 60) if conv.ms2kt(tas) > 0 else 0
        
        return {
            "FL": flight_level,
            "高度(ft)": altitude_ft,
            "速度模式": speed_mode,
            "马赫数": M,
            "CAS(kt)": conv.ms2kt(cas),
            "TAS(kt)": conv.ms2kt(tas),
            "下降率(ft/min)": idle_descent_rate * 196.85,
            "下降率(m/s)": idle_descent_rate,
            "下降角(度)": descent_angle,
            "下降梯度(%)": descent_gradient,
            "高距离(ft/nm)": ft_per_nm,
            "ESF": esf_des,
            "阻力(N)": drag,
            "怠速推力(N)": idle_thrust,
            "燃油流量(kg/h)": fuel_flow_kgh,
            "燃油流量(kg/s)": fuel_flow
        }
    
    # 高度采样 - 每5个高度层一个点
    flight_levels = []
    for fl in range(cruise_fl, target_fl-1, -5):
        flight_levels.append(fl)
    if target_fl not in flight_levels:
        flight_levels.append(target_fl)
    
    results = []
    for fl in flight_levels:
        result = calculate_performance(fl, aircraft_mass)
        results.append(result)
    
    # 创建DataFrame展示结果
    df = pd.DataFrame(results)
    
    # 计算水平距离(累积)和燃油消耗
    horizontal_distance = np.zeros(len(flight_levels))
    fuel_consumption = np.zeros(len(flight_levels))
    
    for i in range(1, len(flight_levels)):
        # 计算相邻高度层之间的高度差(英尺)
        altitude_diff = (flight_levels[i-1] - flight_levels[i]) * 100
        # 使用两个点的平均高距离比
        avg_ft_per_nm = (df.iloc[i-1]['高距离(ft/nm)'] + df.iloc[i]['高距离(ft/nm)']) / 2
        if avg_ft_per_nm > 0:
            segment_distance = altitude_diff / avg_ft_per_nm
        else:
            segment_distance = 0
        # 累积水平距离
        horizontal_distance[i] = horizontal_distance[i-1] + segment_distance
    
    # 计算下降时间和燃油消耗
    descent_times = np.zeros(len(flight_levels))
    total_time_seconds = 0  # 秒
    total_fuel = 0  # kg
    
    for i in range(1, len(flight_levels)):
        # 获取这一段的水平距离
        segment_distance = horizontal_distance[i] - horizontal_distance[i-1]
        
        # 使用平均TAS(kt)，转换为每秒海里数
        avg_tas_per_sec = (df.iloc[i-1]['TAS(kt)'] + df.iloc[i]['TAS(kt)']) / (2 * 3600)
        
        # 计算这一段的时间(秒)
        if avg_tas_per_sec > 0:
            segment_time_seconds = segment_distance / avg_tas_per_sec
        else:
            segment_time_seconds = 0
        
        # 使用平均燃油流量(kg/s)
        avg_fuel_flow = (df.iloc[i-1]['燃油流量(kg/s)'] + df.iloc[i]['燃油流量(kg/s)']) / 2
        
        # 计算这一段的燃油消耗(kg)
        segment_fuel = avg_fuel_flow * segment_time_seconds
        
        # 累积时间和燃油
        total_time_seconds += segment_time_seconds
        total_fuel += segment_fuel
        descent_times[i] = total_time_seconds
        fuel_consumption[i] = fuel_consumption[i-1] + segment_fuel
    
    # 计算总高度变化
    total_altitude_change = (cruise_fl - target_fl) * 100
    
    # 计算平均高距比
    avg_ft_per_nm = total_altitude_change / horizontal_distance[-1] if horizontal_distance[-1] > 0 else 0
    
    # 创建结果摘要
    return {
        "Profile": profile_name,
        "Descent Distance (nm)": round(horizontal_distance[-1], 1),
        "Descent Time (s)": round(total_time_seconds, 0),
        "Descent Time (min)": round(total_time_seconds/60, 1),
        "Fuel Consumption (kg)": round(total_fuel, 1),
        "Average Fuel Flow (kg/h)": round(total_fuel/(total_time_seconds/3600), 1),
        "Average Descent Gradient (%)": round(total_altitude_change/6076.12/horizontal_distance[-1]*100, 2),
        "Altitude-to-Distance Ratio (ft/nm)": round(avg_ft_per_nm, 1)
    }


def compare_descent_profiles(profiles=None, aircraft_model="A320-232", aircraft_mass=62000, cruise_fl=370, target_fl=30, 
                            cruise_mach=0.78, bada_version="4.2", 
                            filepath="/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"):
    """比较多种下降剖面
    
    参数:
    profiles: 下降剖面参数列表，如果为None则使用默认配置
    aircraft_model: 飞机型号
    aircraft_mass: 飞机重量 (kg)
    cruise_fl: 巡航高度层 (FL)
    target_fl: 目标高度层 (FL)
    cruise_mach: 巡航马赫数
    bada_version: BADA版本
    filepath: BADA数据文件路径
    """
    
    # 计算所有剖面
    results = []
    for profile in profiles:
        # 设置默认值
        profile.setdefault("cruise_fl", cruise_fl)
        profile.setdefault("target_fl", target_fl)
        profile.setdefault("aircraft_mass", aircraft_mass)
        profile.setdefault("cruise_mach", cruise_mach)
        profile.setdefault("constant_cas", None)
        profile.setdefault("ac_model", aircraft_model)
        profile.setdefault("bada_version", bada_version)
        profile.setdefault("filepath", filepath)
        
        # 计算下降剖面
        result = calculate_descent_profile(**profile)
        results.append(result)
    
    # 创建比较表格
    df = pd.DataFrame(results)
    
    # 设置表格格式
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.options.display.float_format = '{:.1f}'.format
    
    # 按下降距离排序
    df = df.sort_values(by="Descent Distance (nm)")
    
    # 打印比较表格
    print(f"{aircraft_model} Descent Profiles (FL{cruise_fl} to FL{target_fl}, {aircraft_mass/1000:.1f}ton):")
    # print("=" * 100)
    # print(df[["剖面", "下降距离(nm)", "下降时间(分钟)", "燃油消耗(kg)", "平均燃油流量(kg/h)", "平均下降梯度(%)", "高距比(ft/nm)"]])
    
    # 返回DataFrame以便进一步分析
    return df

In [11]:
profiles_320 = [
            # 马赫数和高高度CAS组合，全部在FL100转为250节
            {"descent_mach": 0.80, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.80, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.79, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.79, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.78, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.78, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.76, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.76, "high_cas": 280, "low_cas": 250, "low_cas_fl": 100},
            
            # 标准低速下降剖面(单一CAS)
            {"descent_mach": 0.73, "high_cas": 250},
            {"descent_mach": 0.72, "high_cas": 250},
            {"descent_mach": 0.71, "high_cas": 250},
            {"descent_mach": 0.70, "high_cas": 250},
            
            # 马赫数/高CAS/低CAS组合(FL150开始)
            {"descent_mach": 0.73, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.72, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.71, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.70, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            
            # 恒定CAS下降
            {"descent_mach": 0.90, "high_cas": 230, "constant_cas": 230},
            {"descent_mach": 0.90, "high_cas": 230, "constant_cas": 230, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.90, "high_cas": 220, "constant_cas": 220}
        ]

compare_descent_profiles(
    profiles=profiles_320, 
    aircraft_model="A320-232", 
    aircraft_mass=60000, 
    cruise_fl=370, 
    target_fl=30
)

A320-232 Descent Profiles (FL370 to FL30, 60.0ton):


,Profile,Descent Distance (nm),Descent Time (s),Descent Time (min),Fuel Consumption (kg),Average Fuel Flow (kg/h),Average Descent Gradient (%),Altitude-to-Distance Ratio (ft/nm)
0,0.8M/310kt/250kt@FL100,88.2,853.0,14.2,137.0,578.3,6.3,385.5
2,0.79M/310kt/250kt@FL100,88.6,858.0,14.3,137.7,577.7,6.3,383.8
4,0.78M/310kt/250kt@FL100,88.9,862.0,14.4,138.2,577.1,6.3,382.5
1,0.8M/300kt/250kt@FL100,92.4,903.0,15.1,145.4,579.5,6.1,367.9
6,0.76M/300kt/250kt@FL100,92.6,912.0,15.2,146.4,577.8,6.0,367.3
3,0.79M/300kt/250kt@FL100,92.7,907.0,15.1,145.9,579.0,6.0,366.9
5,0.78M/300kt/250kt@FL100,92.8,910.0,15.2,146.2,578.5,6.0,366.4
7,0.76M/280kt/250kt@FL100,101.5,1027.0,17.1,165.7,580.9,5.5,335.1
11,0.7M/250kt,113.3,1231.0,20.5,200.3,586.0,4.9,300.1
10,0.71M/250kt,113.6,1232.0,20.5,200.5,586.0,4.9,299.2


## 787

In [10]:
profiles_787 = [
            # 马赫数和高高度CAS组合，全部在FL100转为250节
            {"descent_mach": 0.86, "high_cas": 340, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.86, "high_cas": 330, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.86, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.86, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 340, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 330, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100}, 


            # ECON
            {"descent_mach": 0.84, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},              
            
            # 马赫数/高CAS/低CAS组合(FL150开始)
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},

            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},


            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            
        ]

compare_descent_profiles(
    profiles=profiles_787, 
    aircraft_model="B788GE70", 
    aircraft_mass= 168000,    # MLW: 172,365   B788GE70
    cruise_fl=390, 
    target_fl=30
)

B788GE70下降剖面比较 (FL390到FL30, 168.0吨):


,剖面,下降距离(nm),下降时间(秒),下降时间(分钟),燃油消耗(kg),平均燃油流量(kg/h),平均下降梯度(%),高距比(ft/nm)
0,0.86M/340kt/250kt@FL100,102.2,927.0,15.5,274.7,1066.6,5.8,352.2
4,0.85M/340kt/250kt@FL100,103.0,936.0,15.6,278.2,1070.5,5.8,349.4
1,0.86M/330kt/250kt@FL100,106.0,969.0,16.1,288.0,1070.1,5.6,339.6
5,0.85M/330kt/250kt@FL100,106.7,976.0,16.3,291.1,1073.6,5.5,337.5
2,0.86M/320kt/250kt@FL100,110.2,1017.0,16.9,303.7,1075.3,5.4,326.6
6,0.85M/320kt/250kt@FL100,110.7,1022.0,17.0,306.2,1078.4,5.3,325.2
8,0.84M/310kt/250kt@FL100,114.8,1074.0,17.9,324.8,1088.2,5.2,313.6
3,0.86M/310kt/250kt@FL100,114.9,1071.0,17.9,322.1,1082.6,5.2,313.4
7,0.85M/310kt/250kt@FL100,115.2,1075.0,17.9,324.1,1085.1,5.1,312.5
15,0.79M/250kt,146.2,1532.0,25.5,497.7,1169.6,4.0,246.3


## 飞机类型: B738W26
最大起飞重量 (MTOW): 79,016 kg
最大着陆重量 (MLW): 66,361 kg
参考面积: 124.6 m²
空重: 41,413 kg
最大零燃料重量: 62,732 kg

In [14]:
profiles_737 = [
            # 马赫数和高高度CAS组合，全部在FL100转为250节
            {"descent_mach": 0.80, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.80, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.79, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.79, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.78, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.78, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.76, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.76, "high_cas": 280, "low_cas": 250, "low_cas_fl": 100},
            
            # 标准低速下降剖面(单一CAS)
            {"descent_mach": 0.73, "high_cas": 250},
            {"descent_mach": 0.72, "high_cas": 250},
            {"descent_mach": 0.71, "high_cas": 250},
            {"descent_mach": 0.70, "high_cas": 250},
            
            # 马赫数/高CAS/低CAS组合(FL150开始)
            {"descent_mach": 0.73, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.72, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.71, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.70, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            
            # 恒定CAS下降
            {"descent_mach": 0.90, "high_cas": 230, "constant_cas": 230},
            {"descent_mach": 0.90, "high_cas": 230, "constant_cas": 230, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.90, "high_cas": 220, "constant_cas": 220}
        ]

compare_descent_profiles(
    profiles=profiles_737, 
    aircraft_model="B738W26", 
    aircraft_mass= 62000,    # MLW: 66,361 
    cruise_fl=390, 
    target_fl=30
)

B738W26下降剖面比较 (FL390到FL30, 62.0吨):


,剖面,下降距离(nm),下降时间(秒),下降时间(分钟),燃油消耗(kg),平均燃油流量(kg/h),平均下降梯度(%),高距比(ft/nm)
0,0.8M/310kt/250kt@FL100,97.1,926.0,15.4,153.8,598.2,6.1,370.6
2,0.79M/310kt/250kt@FL100,97.5,931.0,15.5,154.6,597.9,6.1,369.1
4,0.78M/310kt/250kt@FL100,97.8,936.0,15.6,155.4,597.7,6.1,368.1
1,0.8M/300kt/250kt@FL100,100.9,973.0,16.2,161.5,597.4,5.9,356.7
3,0.79M/300kt/250kt@FL100,101.2,977.0,16.3,162.1,597.2,5.9,355.8
6,0.76M/300kt/250kt@FL100,101.2,985.0,16.4,163.6,597.7,5.9,355.8
5,0.78M/300kt/250kt@FL100,101.4,981.0,16.4,162.7,597.2,5.8,355.2
7,0.76M/280kt/250kt@FL100,109.1,1091.0,18.2,180.6,595.7,5.4,330.1
11,0.7M/250kt,118.7,1277.0,21.3,210.4,593.4,5.0,303.4
10,0.71M/250kt,119.0,1277.0,21.3,210.4,593.2,5.0,302.6


飞机类型: B773RR92
最大起飞重量 (MTOW): 299,370 kg
最大着陆重量 (MLW): 237,680 kg
参考面积: 427.8 m²
空重: 157,800 kg
最大零燃料重量: 224,530 kg

In [12]:
profiles_777 = [
            # 马赫数和高高度CAS组合，全部在FL100转为250节
            {"descent_mach": 0.85, "high_cas": 330, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.84, "high_cas": 330, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.84, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.84, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.84, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100}, 


            # ECON
            {"descent_mach": 0.84, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},              
            
            # 马赫数/高CAS/低CAS组合(FL150开始)
            {"descent_mach": 0.80, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.80, "high_cas": 260, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.80, "high_cas": 250},
            {"descent_mach": 0.80, "high_cas": 260},

            
        ]

compare_descent_profiles(
    profiles=profiles_777, 
    aircraft_model="B773RR92", 
    aircraft_mass= 210000,    # MLW: 237,680
    cruise_fl=390, 
    target_fl=30
)

B773RR92 Descent Profiles (FL390 to FL30, 210.0ton):


,Profile,Descent Distance (nm),Descent Time (s),Descent Time (min),Fuel Consumption (kg),Average Fuel Flow (kg/h),Average Descent Gradient (%),Altitude-to-Distance Ratio (ft/nm)
0,0.85M/330kt/250kt@FL100,107.9,986.0,16.4,320.3,1168.8,5.5,333.6
4,0.84M/330kt/250kt@FL100,108.2,991.0,16.5,321.2,1167.4,5.5,332.9
5,0.84M/320kt/250kt@FL100,111.4,1032.0,17.2,334.6,1167.5,5.3,323.1
1,0.85M/320kt/250kt@FL100,111.8,1033.0,17.2,335.0,1167.7,5.3,321.9
6,0.84M/310kt/250kt@FL100,115.4,1081.0,18.0,350.4,1166.4,5.1,311.9
8,0.84M/310kt/250kt@FL100,115.4,1081.0,18.0,350.4,1166.4,5.1,311.9
2,0.85M/310kt/250kt@FL100,116.0,1084.0,18.1,351.1,1166.2,5.1,310.4
7,0.84M/300kt/250kt@FL100,119.5,1136.0,18.9,367.5,1165.0,5.0,301.2
3,0.85M/300kt/250kt@FL100,120.2,1140.0,19.0,368.6,1164.3,4.9,299.4
12,0.8M/260kt,132.3,1353.0,22.6,436.1,1160.3,4.5,272.1


[10] ✓ : A350-941
-------------------------------------------------- 
飞机类型: A350-941
最大起飞重量 (MTOW): 275,000 kg
最大着陆重量 (MLW): 207,000 kg
参考面积: 443.0 m²
空重: 146,596 kg
最大零燃料重量: 195,700 kg
MMO: 0.89
VMO: 340.0

In [ ]:
profiles_350 = [
            # 马赫数和高高度CAS组合，全部在FL100转为250节
            {"descent_mach": 0.86, "high_cas": 340, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.86, "high_cas": 330, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.86, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.86, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 340, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 330, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},
            {"descent_mach": 0.85, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100}, 


            # ECON
            {"descent_mach": 0.84, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100}, 
            {"descent_mach": 0.84, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100}, 
            {"descent_mach": 0.84, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},   
  

            {"descent_mach": 0.80, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.80, "high_cas": 260, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.80, "high_cas": 250},
            {"descent_mach": 0.80, "high_cas": 260},
           
            
            # 马赫数/高CAS/低CAS组合(FL150开始)
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},

            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},


            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},


            {"descent_mach": 0.78, "high_cas": 250},
            {"descent_mach": 0.78, "high_cas": 250},
            {"descent_mach": 0.78, "high_cas": 250},
            {"descent_mach": 0.78, "high_cas": 250},




            
        ]

compare_descent_profiles(
    profiles=profiles_350, 
    aircraft_model="A350-941", 
    aircraft_mass= 190000,    # MLW:207,000  A350
    cruise_fl=390, 
    target_fl=30
)

A350-941 Descent Profiles (FL390 to FL30, 190.0ton):


,Profile,Descent Distance (nm),Descent Time (s),Descent Time (min),Fuel Consumption (kg),Average Fuel Flow (kg/h),Average Descent Gradient (%),Altitude-to-Distance Ratio (ft/nm)
0,0.86M/340kt/250kt@FL100,100.3,907.0,15.1,357.3,1418.5,5.9,359.1
4,0.85M/340kt/250kt@FL100,100.7,912.0,15.2,358.5,1414.7,5.9,357.5
1,0.86M/330kt/250kt@FL100,104.0,948.0,15.8,371.4,1409.9,5.7,346.2
5,0.85M/330kt/250kt@FL100,104.3,953.0,15.9,372.3,1406.9,5.7,345.2
10,0.84M/320kt/250kt@FL100,107.7,996.0,16.6,386.4,1396.3,5.5,334.4
2,0.86M/320kt/250kt@FL100,108.0,995.0,16.6,386.9,1400.0,5.5,333.4
6,0.85M/320kt/250kt@FL100,108.2,998.0,16.6,387.5,1397.8,5.5,332.9
9,0.84M/310kt/250kt@FL100,111.6,1045.0,17.4,402.7,1387.1,5.3,322.5
3,0.86M/310kt/250kt@FL100,112.2,1046.0,17.4,403.7,1389.2,5.3,320.8
7,0.85M/310kt/250kt@FL100,112.3,1048.0,17.5,404.0,1387.7,5.3,320.7


✓ 
飞机类型: A330-341
最大起飞重量 (MTOW): 212,000 kg
最大着陆重量 (MLW): 174,000 kg
参考面积: 361.6 m²
空重: 126,000 kg
最大零燃料重量: 164,000 kg
MMO: 0.86
VMO: 330.0

In [20]:
profiles_330 = [
            # 马赫数和高高度CAS组合，全部在FL100转为250节
           

            # ECON
            {"descent_mach": 0.83, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100}, 
            {"descent_mach": 0.83, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100}, 
            {"descent_mach": 0.83, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},   


            {"descent_mach": 0.82, "high_cas": 300, "low_cas": 250, "low_cas_fl": 100}, 
            {"descent_mach": 0.82, "high_cas": 310, "low_cas": 250, "low_cas_fl": 100}, 
            {"descent_mach": 0.82, "high_cas": 320, "low_cas": 250, "low_cas_fl": 100},   
  
  

        
            
            # 马赫数/高CAS/低CAS组合(FL150开始)
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.79, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},

            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},
            {"descent_mach": 0.79, "high_cas": 250},


            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},
            {"descent_mach": 0.78, "high_cas": 250, "low_cas": 220, "low_cas_fl": 150},


            {"descent_mach": 0.78, "high_cas": 250},
            {"descent_mach": 0.78, "high_cas": 250},
            {"descent_mach": 0.78, "high_cas": 250},
            {"descent_mach": 0.78, "high_cas": 250},

            
        ]

compare_descent_profiles(
    profiles=profiles_330, 
    aircraft_model="A330-341", 
    aircraft_mass= 170000,    # MLW:174,000 
    cruise_fl=390, 
    target_fl=30
)

A330-341 Descent Profiles (FL390 to FL30, 170.0ton):


,Profile,Descent Distance (nm),Descent Time (s),Descent Time (min),Fuel Consumption (kg),Average Fuel Flow (kg/h),Average Descent Gradient (%),Altitude-to-Distance Ratio (ft/nm)
2,0.83M/320kt/250kt@FL100,110.5,1028.0,17.1,327.4,1146.7,5.4,325.8
5,0.82M/320kt/250kt@FL100,110.9,1034.0,17.2,328.6,1144.3,5.3,324.5
1,0.83M/310kt/250kt@FL100,114.7,1079.0,18.0,340.6,1136.5,5.2,313.9
4,0.82M/310kt/250kt@FL100,115.0,1083.0,18.1,341.4,1134.7,5.2,313.1
0,0.83M/300kt/250kt@FL100,119.1,1135.0,18.9,354.9,1125.8,5.0,302.2
3,0.82M/300kt/250kt@FL100,119.3,1138.0,19.0,355.4,1124.5,5.0,301.9
18,0.78M/250kt,139.6,1475.0,24.6,439.6,1072.6,4.2,257.9
19,0.78M/250kt,139.6,1475.0,24.6,439.6,1072.6,4.2,257.9
20,0.78M/250kt,139.6,1475.0,24.6,439.6,1072.6,4.2,257.9
21,0.78M/250kt,139.6,1475.0,24.6,439.6,1072.6,4.2,257.9
